<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="Rossum"></p>

In [1]:
# import libraries

In [2]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.preprocessing import image
import matplotlib.pyplot as plt
from matplotlib.image import imread
import warnings
warnings.filterwarnings('ignore')

The Dogs vs. Cats dataset is a common computer vision dataset in which pictures are classified as either including a dog or a cat.

After the dataset is well studied, it can be used to understand and practice how to design, evaluate, and apply convolutional neural networks for image classification.

You will build a classifier with images and try to detect dogs versus cats using CNN.

Train set includes 12500 cat-5026 dog images, validation set includes 1219 cat-1071 dog images and test set incgludes 6897 cat and dogs images together. 

In [3]:
# read data and check how many class in train set

In [4]:
pwd

In [5]:
my_data_dir = '../input/dog-cat/data'

In [6]:
os.listdir(my_data_dir)

In [7]:
test_path = my_data_dir+'/test/'
train_path = my_data_dir+'/train/'
val_path = my_data_dir+'/validation/'

In [8]:
os.listdir(test_path)

In [9]:
os.listdir(train_path)

**Let's check how many images there are.**

In [10]:
len(os.listdir(train_path+'cat'))

In [11]:
#number of cat images in train set

In [12]:
len(os.listdir(train_path+'dog'))

In [13]:
#number of dog images in train set

In [14]:
#take cat image at index 0 from train set
#take dog image at index 1 from train set
#compare their shapes


In [15]:
os.listdir(train_path+'cat')[0]

In [16]:
cat_cell = train_path+'cat'+'/cat.0.jpg'

In [17]:
os.listdir(train_path+'dog')[1]

In [18]:
dog_cell = train_path+'dog'+'/2464.jpg'

In [19]:
imread(cat_cell)

In [20]:
cat_img = imread(cat_cell)

In [21]:
plt.imshow(cat_img)

In [22]:
imread(dog_cell)

In [23]:
dog_img= imread(dog_cell)

In [24]:
plt.imshow(dog_img)

**Let's decide on the final dimension of these images.**

In [27]:
cat_img.shape

In [28]:
dog_img.shape

In [29]:
# decide on a shape to use it in your model as input shape. you can either use average shape size or try something different. 128*128 is preferred here
image_shape = (128,128,3)

In [30]:
 # help(ImageDataGenerator)

**Image Manipulation**

We can use the ImageDataGenerator to manipulate the images with rotation, resizing, and scaling so the model becomes more robust to different images that our data set doesn't have. ImageDataGenerator does the followings.

* Accepts a batch of images used for training.
* Applies a series of random transformations to each image in the batch.
* Replaces the original batch with randomly transformed batch.
* Training the CNN on this randomly transformed batch.

The goal of applying data augmentation is to have a more generalized model.

In [31]:
#use ImageDataGenerator to manipulate the images

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
image_gen =ImageDataGenerator(rotation_range=15, 
                               width_shift_range=0.10, 
                               height_shift_range=0.10, 
                               #rescale=1/255, 
                               shear_range=0.1, 
                               zoom_range=0.1, 
                               horizontal_flip=True, 
                               fill_mode='nearest')

In [34]:
plt.imshow(image_gen.random_transform(dog_img))

In [35]:
#show transfromed version of dog image

### Generating many manipulated images from a directory


flow_from_directory function works with images organized in sub-directories. Your directories should include only one class of images, so one folder per class of images.

main_directory/

...class_a/

......a_image_1.jpg

......a_image_2.jpg

...class_b/

......b_image_1.jpg

......b_image_2.jpg

[bağlantı metni](https://drive.google.com/file/d/1ntiw1WnsWq5en96zRQmWADFxoCKS4DNc/view?usp=sharing)

In [36]:
image_gen.flow_from_directory(val_path)

image_gen.flow_from_directory(train_path)

In [37]:
image_gen.flow_from_directory(val_path)

# Creating the Model

In [38]:
#create your CNN model here

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [40]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
model.summary()

## Early Stopping

In [42]:
from tensorflow.keras.callbacks import EarlyStopping

In [43]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [44]:
batch_size = 16

In [45]:
#use image_gen from above and read iamges from train and test set

In [46]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [62]:
test_image_gen = image_gen.flow_from_directory(val_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [63]:
val_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

In [48]:
#show class indices in train set

In [49]:
train_image_gen.class_indices

## Model Training

In [50]:
import warnings
warnings.filterwarnings('ignore')

In [52]:
results = model.fit_generator(train_image_gen,epochs=10,
                              validation_data=val_image_gen, callbacks=[early_stop])

In [54]:
summary = pd.DataFrame(model.history.history)
summary.head()

# Evaluating the Model

In [56]:
losses = pd.DataFrame(summary.loss, summary.val_loss)

In [57]:
plt.figure(figsize=(10,6))
plt.plot(summary.loss, label="loss")
plt.plot(summary.val_loss, label="val_loss")
plt.legend(loc="upper right")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [58]:
model.metrics_names

In [64]:
model.evaluate_generator(test_image_gen)

In [65]:
from tensorflow.keras.preprocessing import image

In [73]:
pred_probabilities = model.predict_generator(test_image_gen)

In [67]:
pred_probabilities

In [68]:
pred_probabilities

In [69]:
test_image_gen.classes

In [70]:
predictions = pred_probabilities > 0.5

In [71]:
predictions

In [72]:
predictions

In [74]:
from sklearn.metrics import classification_report,confusion_matrix

In [76]:
print(classification_report(test_image_gen.classes,predictions))

In [77]:
confusion_matrix(test_image_gen.classes,predictions)

In [79]:
from tensorflow.keras.models import load_model
model.save('cat_dog_detector.h5')

# Predicting on an Image

In [81]:
from PIL import Image

In [ ]:
#read 2749.jpg from test set

In [83]:
my_image = image.load_img(dog_cell,target_size=image_shape)

In [84]:
my_image

In [85]:
type(my_image)

In [87]:
my_image.shape

In [88]:
my_image = np.expand_dims(my_image, axis=0)

In [89]:
my_image.shape

In [90]:
model.predict(my_image)

In [91]:
train_image_gen.class_indices

In [92]:
test_image_gen.class_indices

## THE END